LL parser using First and Follow for grammar

S -> S '|' S | A 

A -> A ',' A | P

P -> P '*' | 'chr' | 'nil'  :

As the result first set and follow will be printed and parsing table too.

In [15]:
class FirstFollow:
    def __init__(self):
        self.gram={'S':['|SS', 'A'],'A':[',AA','P'], 'P':['*P', 'c', 'n']}
        self.term=[',','*', '|','(',')','c','n']
        self.nonterm=['S','A','P']
          
    def first(self,ip):
        fir=[]
        ctr=0
        length=0
        if(ip in self.term):
            fir.extend(ip)
        else:
            for i in self.gram[ip]:
                if(i[0] in self.term):
                     fir.extend(i[0])
                else:
                     length=len(i)
                     while(ctr<length):
                          if('n' in self.gram[i[ctr]]):
                                fir.extend(self.first(i[ctr]))
                                ctr+=1
                          else:
                                fir.extend(self.first(i[ctr]))
                                break
        firstset[ip]=fir
        return fir

    def follow(self,ip):
        foll=[]
        if(ip=='E'):
            foll.extend('$')
        for key in self.gram.keys():
            vals=self.gram[key]
            for each in vals:
                ctr=0
                length=len(each)
                for j in each:
                    if(j==ip):
                        if(ctr<length-1):
                            if((ip != key)and('n'in self.first(each[ctr+1]))):
                                for x in self.first(each[ctr+1]):
                                    if((x not in foll)and(x!='n')):
                                        foll.extend(x)
                                for x in self.follow(key):
                                    if((x not in foll)and(x!='n')):
                                        foll.extend(x)
                            else:
                                for x in self.first(each[ctr+1]):
                                    if((x not in foll)and(x!='n')):
                                        foll.extend(x)
                        if((ip != key)and(ctr==length-1)):
                            for x in self.follow(key):
                                if((x not in foll)and(x!='n')):
                                    foll.extend(x)
                    ctr+=1
                ctr=0
        followset[ip]=foll
        return foll
      
    def parsingtable(self,ip):
        for i in self.gram[ip]:   
            if ip not in parsingtable: 
                parsingtable[ip]={}  
            if i[0] in self.term and i[0]!='n':
                    #print(i)
                if i[0] not in parsingtable[ip]:
                    parsingtable[ip][i[0]]=[]
                parsingtable[ip][i[0]].append(str(ip +" -> "+ i))
            elif i == 'n':
                print("2")
                for k in followset[ip]:
                    if k not in parsingtable[ip]: 
                        parsingtable[ip][k]=[]
                    parsingtable[ip][k].append(str(ip +" -> "+ i))
            else:
                print("3")
                for k in firstset[ip]:
                    if k not in parsingtable[ip]: 
                        parsingtable[ip][k]=[]
                    parsingtable[ip][k].append(str(ip + " -> "+i))
                
    def printparser(self):
        print("Parsing table")
        for i in parsingtable:
            for j in parsingtable[i]:
                for k in parsingtable[i][j]:
                    print(i,":",j,":",k) 
firstset={}
followset={}
parsingtable={}
a=FirstFollow()
nont=['S','A','P']
for i in nont:
    fi=a.first(i)
    fo=a.follow(i)
    firstset[i]=fi
    followset[i]=fo
    

print(firstset)
print(followset)
for i in nont:
    a.parsingtable(i)

a.printparser()


{'P': ['*', 'c', 'n'], 'A': [',', '*', 'c', 'n'], 'S': ['|', ',', '*', 'c', 'n']}
{'S': ['|', ',', '*', 'c'], 'A': ['|', ',', '*', 'c'], 'P': ['|', ',', '*', 'c']}
3
3
2
Parsing table
S : | : S -> |SS
S : | : S -> A
S : , : S -> A
S : * : S -> A
S : c : S -> A
S : n : S -> A
A : , : A -> ,AA
A : , : A -> P
A : * : A -> P
A : c : A -> P
A : n : A -> P
P : * : P -> *P
P : * : P -> n
P : c : P -> c
P : c : P -> n
P : | : P -> n
P : , : P -> n



This program is an example of syntax directed translation for assembler.
Assambley class is an abstract class that provides functions
     for assembly language code. 
Production Rules Translation Schemes (|| is concatenation)

Expr → Term Expr’ Expr.val = Term.val || Expr’.val

Expr’ → + Term Expr’ Expr’.val = Term.val || Expr’.val || ASM Code for +

Expr’ → – Term Expr’ Expr’.val = Term.val || Expr’.val || ASM Code for –

Expr’ → ϵ

Term → Factor Term’ Term.val = Factor.val || Term’.val

Term’ → * Factor Term’ Term’.val = Factor.val || Term’.val || ASM Code for *

Term’ → / Factor Term’ Term’.val = Factor.val || Term’.val || ASM Code for /

Term’ → % Factor Term’ Term’.val = Factor.val || Term’.val || ASM Code for %

Term’ → ϵ

Factor → ( Expr ) Factor.val = Expr.val

Factor → Number Factor.val = ASM Code for Number

In [14]:
import re, abc, sys

     
class Assembly(abc.ABC):
    
    @abc.abstractmethod
    def get_assembly_code(self, character):
        pass
    
    def generate_start_assembly(self):
        
        return (".CODE\n"
                "MAIN PROC\n\n"
                "    MOV EAX,0\n")
    
    def generate_end_code(self):
        
        return ("    CALL WRITEDEC\n\n"
                "MAIN ENDP\n"
                "END MAIN")
        

class SDT(Assembly):
    
    def __init__(self, expression):
        
        self.tokens_list = self.generate_tokens(expression)
        self.index = 0
        
    def generate_tokens(self, expression):
        return re.findall('[0-9]+|\+|\-|\/|\*|\(|\)', expression)
    
    def __move_next__(self):
        if self.index < len(self.tokens_list) - 1:
            self.index += 1
    
    
    def display_tokens_list(self):
        
        print()
        for i in range(len(self.tokens_list)): 
            
            print("Token[" + str(i) + "]  =  " + self.tokens_list[i], end = "")    
            print("  (Number)") if self.tokens_list[i].isdigit() else print(" (Operator)")
    
        print("\n")
    def get_assembly_code(self, character):
        
        switch_case = {
                
                "+" : "    POP AX\n"
                      "    POP BX\n"
                      "    ADD AX,BX\n"
                      "    PUSH AX\n",
                    
                "-" : "    POP BX\n"
                      "    POP AX\n"
                      "    SUB AX,BX\n"
                      "    PUSH AX\n",
                    
                "*" : "    POP AX\n"
                      "    POP BX\n"
                      "    MUL BX\n"
                      "    PUSH AX\n",
                
                "/" : "    POP BX\n"
                      "    POP AX\n"
                      "    MOV DX, 0\n"
                      "    DIV BX\n"
                      "    PUSH AX\n",
                     
                "%" : "    POP BX\n"
                      "    POP AX\n"
                      "    MOV DX, 0\n"
                      "    DIV BX\n"
                      "    PUSH DX\n"
        }
    
        return "    MOV AX, " + character + "\n    PUSH AX\n" if character.isdigit() else switch_case.get(character, "")
    
    
    def __str__(self):
        print("Parse Tree Leaf Nodes: ")
        file_text = super().generate_start_assembly() + self.__expr() + super().generate_end_code()
        print("\n\nAssembly file has been successfully generated with the following code, named: output.asm\n\n\n")
       # print(file_text)
        with open('output.asm', 'w') as fout:
            fout.write(file_text)
            
        return file_text
        
    
    def __expr(self):
        return self.__term() + self.__expr__()

    def __expr__(self):
        
        if self.tokens_list[self.index] == "+" or self.tokens_list[self.index] == "-":
            
            current_index_assembly = self.get_assembly_code(self.tokens_list[self.index])
            self.__move_next__()
            return self.__term() + self.__expr__() + current_index_assembly
        
        else:
            print("\u03B5 ")
            return ""
            
            
    def __term(self):
        return self.__factor() + self.__term__()
    
    def __factor(self):
        
        if self.tokens_list[self.index].isdigit():
            
            current_index_assembly = self.get_assembly_code(str(self.tokens_list[self.index]))
            print(self.tokens_list[self.index])
            self.__move_next__()
            
            return current_index_assembly
        
        elif self.tokens_list[self.index] == "(":
    
            print(self.tokens_list[self.index])
            self.__move_next__()
            expr_val = self.__expr()
            
            if self.tokens_list[self.index] == ")":
                print(self.tokens_list[self.index])
                self.__move_next__() 

            return expr_val    
    
    def __term__(self):
        
        if self.tokens_list[self.index] == "*" or self.tokens_list[self.index] == "/" or self.tokens_list[self.index] == "%":
            
            
            print(self.tokens_list[self.index])
            current_index_assembly = self.get_assembly_code(self.tokens_list[self.index])
            self.__move_next__()
            
            return self.__factor() + self.__term__() + current_index_assembly
        
        else:
            print("\u03B5 ")
            return ""

        
if __name__ == '__main__':
    
        argv =  'Eval /? Eval <>'
        argv1 = 'Eval 22 * (50 + (36 / 12) - 16)'
        expression = argv1
        sdt = SDT(expression)
        sdt.display_tokens_list()
        print(sdt)
        
        #print("\n\nExpected number of arguments: 2(1-python file\t2-expression in string), found: " + str(len(sys.argv)) + " arguments, terminating...\n\n")
    


Token[0]  =  22  (Number)
Token[1]  =  * (Operator)
Token[2]  =  ( (Operator)
Token[3]  =  50  (Number)
Token[4]  =  + (Operator)
Token[5]  =  ( (Operator)
Token[6]  =  36  (Number)
Token[7]  =  / (Operator)
Token[8]  =  12  (Number)
Token[9]  =  ) (Operator)
Token[10]  =  - (Operator)
Token[11]  =  16  (Number)
Token[12]  =  ) (Operator)


Parse Tree Leaf Nodes: 
22
*
(
50
ε 
(
36
/
12
ε 
ε 
)
ε 
16
ε 
ε 
)
ε 
ε 


Assembly file has been successfully generated with the following code, named: output.asm



.CODE
MAIN PROC

    MOV EAX,0
    MOV AX, 22
    PUSH AX
    MOV AX, 50
    PUSH AX
    MOV AX, 36
    PUSH AX
    MOV AX, 12
    PUSH AX
    POP BX
    POP AX
    MOV DX, 0
    DIV BX
    PUSH AX
    MOV AX, 16
    PUSH AX
    POP BX
    POP AX
    SUB AX,BX
    PUSH AX
    POP AX
    POP BX
    ADD AX,BX
    PUSH AX
    POP AX
    POP BX
    MUL BX
    PUSH AX
    CALL WRITEDEC

MAIN ENDP
END MAIN
